In [1]:
from bs4 import BeautifulSoup
from os.path import exists
import numpy as np
import pandas as pd
import requests as req
import time
import os
import lxml
import pickle

import re

In [2]:
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
DATA_ROOT_PATH="/mnt/data/projects/MD1"
PROJ_ROOT_PATH="/home/priyesh/projects/MD1"

In [4]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','ticker_list.pkl')
ticker_list = pd.read_pickle(filepath)

In [5]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','master_data_stage2.pkl')
master_data = pd.read_pickle(filepath)

In [6]:
# Read Company Profile Data 

# This is data downloaded from Yahoo as part of project MD3

filepath = os.path.join(DATA_ROOT_PATH,'data/import','yahoo_company_industry.csv')
company_industry = pd.read_csv(filepath,index_col=False)

filepath = os.path.join(DATA_ROOT_PATH,'data/import','yahoo_drop_company.csv')
drop_company = pd.read_csv(filepath,index_col=False)

In [7]:
drop_list = list(set(list(drop_company['ticker'])))

drop_list = drop_list + list(company_industry[company_industry['yahoo_industry'] == 'Healthcare Plans']['ticker'])
drop_list = drop_list + list(company_industry[company_industry['yahoo_sector'] == 'Real Estate']['ticker'])
drop_list


['LSXMA',
 'NWSA',
 'CXT',
 'CSX',
 'VTS',
 'FOXA',
 'KD',
 'PYCR',
 'CACI',
 'PARAA',
 'BF-A',
 'DRI',
 'PAYX',
 'FDX',
 'UAA',
 'LW',
 'RPM',
 'CTAS',
 'AZPN',
 'HRB',
 'LSXMK',
 'CHK',
 'FYBR',
 'UA',
 'NKE',
 'FWONK',
 'BEPC',
 'UHAL',
 'CAG',
 'CASY',
 'BKI',
 'OSK',
 'DH',
 'DNB',
 'FWONA',
 'ORCL',
 'GIS',
 'LBRDK',
 'ZG',
 'RGLD',
 'CNC',
 'CI',
 'CVS',
 'ELV',
 'HUM',
 'MOH',
 'UNH',
 'ARE',
 'AMH',
 'AMT',
 'COLD',
 'AIRC',
 'AVB',
 'BXP',
 'BRX',
 'CPT',
 'CBRE',
 'CUZ',
 'CCI',
 'CUBE',
 'DLR',
 'DEI',
 'EGP',
 'EPR',
 'EQIX',
 'ELS',
 'EQR',
 'ESS',
 'EXR',
 'FRT',
 'FR',
 'GLPI',
 'HR',
 'PEAK',
 'HIW',
 'HST',
 'HHC',
 'INVH',
 'IRM',
 'JBGS',
 'JLL',
 'KRC',
 'KIM',
 'LAMR',
 'LSI',
 'MPW',
 'MAA',
 'NNN',
 'NSA',
 'OHI',
 'OPEN',
 'PK',
 'PLD',
 'PSA',
 'RYN',
 'O',
 'REG',
 'REXR',
 'SBAC',
 'SPG',
 'SLG',
 'SRC',
 'SUI',
 'UDR',
 'VTR',
 'VICI',
 'VNO',
 'WPC',
 'WELL',
 'WE',
 'WY',
 'CSGP']

In [8]:
for ticker in drop_list:
    
  try:
    ticker_list.remove(ticker)
  except ValueError:
    pass

ticker_list

['AA',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACHC',
 'ACI',
 'ACM',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AES',
 'AFG',
 'AFL',
 'AFRM',
 'AGCO',
 'AGL',
 'AGNC',
 'AGO',
 'AGR',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'AL',
 'ALB',
 'ALGM',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALLY',
 'ALNY',
 'ALSN',
 'AM',
 'AMAT',
 'AMC',
 'AMCR',
 'AMD',
 'AME',
 'AMED',
 'AMG',
 'AMGN',
 'AMP',
 'AMZN',
 'AN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APO',
 'APP',
 'APTV',
 'AR',
 'ARES',
 'ARMK',
 'ARW',
 'ASH',
 'ATO',
 'ATR',
 'ATUS',
 'ATVI',
 'AVGO',
 'AVT',
 'AVTR',
 'AWI',
 'AWK',
 'AXON',
 'AXS',
 'AXTA',
 'AYI',
 'AYX',
 'AZEK',
 'AZO',
 'AZTA',
 'BA',
 'BAC',
 'BAH',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BC',
 'BDX',
 'BEN',
 'BERY',
 'BFAM',
 'BG',
 'BHF',
 'BIIB',
 'BILL',
 'BIO',
 'BJ',
 'BK',
 'BKNG',
 'BLD',
 'BLDR',
 'BLK',
 'BMRN',
 'BMY',
 'BOKF',
 'BR',
 'BRKR',
 'BRO',
 'BSX',
 'BSY',
 'BURL',
 'BWA',
 'BW

In [9]:
# Remove LIN from the list as it creates errors

ticker_list.remove('LIN')

# Remove GOOGL as it is a duplicate of GOOG

ticker_list.remove('GOOGL')

In [10]:
def extract_tables():
    
  master_data = {}

  for ticker in ticker_list:
    
    print(ticker)
    path = os.path.join(DATA_ROOT_PATH,'data/R1000/reports',ticker)
    dir_list = os.listdir(path)
  
    master_data[ticker] = {}
    
    for yr in dir_list:
      bal_sheet_path = os.path.join(path,yr,'bal_report.htm')
      inc_sheet_path = os.path.join(path,yr,'inc_report.htm')
      cash_sheet_path = os.path.join(path,yr,'cash_report.htm')
    
      master_data[ticker][yr] = {}
    
      if os.path.isfile(bal_sheet_path):
        tables = pd.read_html(bal_sheet_path)
        master_data[ticker][yr]['balance'] = tables[0]
        
      if os.path.isfile(inc_sheet_path):
        tables = pd.read_html(inc_sheet_path)
        master_data[ticker][yr]['income'] = tables[0]
    
      if os.path.isfile(cash_sheet_path):
        tables = pd.read_html(cash_sheet_path)
        master_data[ticker][yr]['cash'] = tables[0]
        
  return master_data

In [11]:
master_data = extract_tables()

AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACGL
ACHC
ACI
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AFRM
AGCO
AGL
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
AMAT
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMP
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APO
APP
APTV
AR
ARES
ARMK
ARW
ASH
ATO
ATR
ATUS
ATVI
AVGO
AVT
AVTR
AWI
AWK
AXON
AXS
AXTA
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAH
BALL
BAX
BBWI
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
BIIB
BILL
BIO
BJ
BK
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BSX
BSY
BURL
BWA
BWXT
BX
BYD
C
CABO
CACC
CAH
CAR
CARR
CAT
CB
CBOE
CBSH
CC
CCCS
CCK
CCL
CDAY
CDNS
CDW
CE
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHPT
CHRW
CHTR
CIEN
CINF
CL
CLH
CLVT
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNM
CNP
CNXC
COF
COIN
COLB
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CR
CRI
CRL
CRUS
CRWD
CSCO
CSL
CTLT
CTRA
CTSH
CTVA
CVNA
CVX
CW
CZR
D
DAL
DAR
DASH
DBX
DCI
DD
DDOG
DECK
DFS
DG
DGX
DHI
DHR
DINO
DIS
DISH
DKNG
DKS
DLB
DLTR
DNA
DOCS
DOCU
DOV
DOW
DPZ
DRVN
DT
DTE
DTM
DUK
DV
DVA
DVN
DXC
DX

In [12]:
df = master_data['CSCO']['15']['income']
df

Consolidated Statements Of Operations - USD ($)  shares in Millions, $ in Millions  \
   Consolidated Statements Of Operations - USD ($)  shares in Millions, $ in Millions   
0                                                                            REVENUE:   
1                                                                             Product   
2                                                                             Service   
3                                                                       Total revenue   
4                                                                      COST OF SALES:   
5                                                                             Product   
6                                                                             Service   
7                                                                 Total cost of sales   
8                                                                        GROSS MARGIN   
9                                                                 OPERATING EXPENSES:   
10                                                           Research and development   
11                                                                Sales and marketing   
12                                                         General and administrative   
13                                        Amortization of purchased intangible assets   
14                                                    Restructuring and other charges   
15                                                           Total operating expenses   
16                                                                   OPERATING INCOME   
17                                                                    Interest income   
18                                                                   Interest expense   
19                                                           Other income (loss), net   
20                                              Interest and other income (loss), net   
21                                           INCOME BEFORE PROVISION FOR INCOME TAXES   
22                                                         Provision for income taxes   
23                                                                         NET INCOME   
24                                                              Net income per share:   
25                                                       Basic (in dollars per share)   
26                                                     Diluted (in dollars per share)   
27                                              Shares used in per-share calculation:   
28                                                                  Basic (in shares)   
29                                                                Diluted (in shares)   
30                    Cash dividends declared per common share (in dollars per share)   

   12 Months Ended                              
     Jul. 25, 2015 Jul. 26, 2014 Jul. 27, 2013  
0              NaN           NaN           NaN  
1         $ 37,750      $ 36,172      $ 38,029  
2            11411         10970         10578  
3            49161         47142         48607  
4              NaN           NaN           NaN  
5            15377         15641         15541  
6             4103          3732          3626  
7            19480         19373         19167  
8            29681         27769         29440  
9              NaN           NaN           NaN  
10            6207          6294          5942  
11            9821          9503          9538  
12            2040          1934          2264  
13             359           275           395  
14             484           418           105  
15           18911         18424         18244  
16           10770          9345         11196  
17             769           691           654  
18           (566)         (564)         (583)  
19             228           243    

In [19]:
# Save to file

with open(PROJ_ROOT_PATH + '/pickle/master_data_stage2.pkl', 'wb') as f:
  pickle.dump(master_data, f)


## Clean and format tables

In [6]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','master_data_stage2.pkl')
master_data = pd.read_pickle(filepath)

In [7]:
def clean_table(orig_df):
    
  df =orig_df.astype(str).copy()

  # Remove multi-levels in coklumn index 

  nlevels = df.columns.nlevels

  cols = list(df.columns)    
    
  if nlevels > 1:
        
    # Find columns identied as 12 months
    
    new_cols=[cols[0]]
    
    for i in range(1,len(cols)):
    
      if '12 Months' in cols[i][0]:
            
        new_cols = new_cols + [cols[i]]
        
    # If we do not find any matches on 12 Months then keep the original columns
    
    if len(new_cols) > 1:
        
      df = df[new_cols].copy()
    
    # Now eliminate the multi-level column index
     
    level = list(range(0,nlevels - 1))  
    cols = list(df.columns.droplevel(level))
    
  else:
    cols = list(df.columns)

  tableHeader = cols[0]

  # Determine whether amounts are in millions or thousands (ignore the "shares in millions/thousands")
  # If we cannot find the text, assume thousands
    
  t = tableHeader
  t = t.strip().replace(" ","").lower().replace("sharesinmillions","")
  t = t.strip().replace(" ","").lower().replace("sharesinthousands","")

  if "millions" in t:
     units = 'M'
  else:
     units = 'T'
    
  # Some headers don't have strings eg. SharesinMillions, so remove spaces and convert to lowercase
    
  t = tableHeader.replace

  # Rename the first column and assign new column names for the remaining columns
    
  cols[0] = 'line_item'
  df.columns = cols

  # Remove apostrophes and quotes

  df['line_item'] = df['line_item'].str.replace("’", "", regex=True)
  df['line_item'] = df['line_item'].str.replace("'", "", regex=True)
    
  # Replace \xa0 (non breaking space) with space

  df = df.replace(to_replace=u'\xa0',value=' ', regex=True)

  # Replace values in paranthesis with negative numeric value

  df = df.replace(to_replace="\(",value="-",regex=True).replace(to_replace="\)",value="",regex=True)
    
  # Remove $ sign
  
  df = df.replace(to_replace="\$",value="",regex=True)

  # Remove non numeric values in columns (except for first column). Also remove comma in numeric values.
  
  for col in cols[1:]:
    df[col] = df[col].astype(str).str.replace(",","")
    df[col] = pd.to_numeric(df[col], errors ='coerce')
    
  # Drop columns that only have nulls

  df = df.dropna(axis=1, how='all')
    
  # Remove footnotes embedded in table
    
  df.drop(df.loc[df[df.columns[0]].str.contains(r"\[\d+\]", case=False, regex=True,na=False)].index,inplace=True)

  # Remove note annotations eg. Sales Revenue -A

  df['line_item'] = df['line_item'].replace(to_replace="-[0-9A-Z]+[:]?$",value="",regex=True)
  df['line_item'] = df['line_item'].replace(to_replace=" [0-9A-Z]?[:]?$",value="",regex=True)
  df['line_item'] = df['line_item'].replace(to_replace="-Note[s]? [0-9A-Z]+.*$",value="",regex=True)
  df['line_item'] = df['line_item'].replace(to_replace="- Note[s]? [0-9A-Z]+.*$",value="",regex=True)
  df['line_item'] = df['line_item'].replace(to_replace="-See Note[s]? [0-9A-Z]+.*$",value="",regex=True)
 
  # General tidy up

  df['line_item'] = df['line_item'].replace(to_replace=' —',value=' ',regex=True)
  df['line_item'] = df['line_item'].replace(to_replace=' -',value=' ',regex=True)

  df['line_item'] = df['line_item'].replace(to_replace='—$',value='',regex=True)
  df['line_item'] = df['line_item'].replace(to_replace='-$',value='',regex=True)
    
  # Strip trailing and leadning spaces

  df['line_item'] = df['line_item'].str.strip()
    
  # Standardise line item descriptions so that each word starts with a capital and rest is lower case.

  df['line_item'] = df['line_item'].str.title()
    
  # Set index to line_item

  df = df.set_index('line_item',drop=True) 

  table = {'tableHeader': tableHeader,
           'table': df,
           'unit': units}
  
  return table

In [8]:
clean_table(master_data['CSCO']['15']['income'])['table']

,"Jul. 25, 2015","Jul. 26, 2014","Jul. 27, 2013"
line_item,,,
Revenue:,NaN,NaN,NaN
Product,37750.00,36172.00,38029.00
Service,11411.00,10970.00,10578.00
Total Revenue,49161.00,47142.00,48607.00
Cost Of Sales:,NaN,NaN,NaN
Product,15377.00,15641.00,15541.00
Service,4103.00,3732.00,3626.00
Total Cost Of Sales,19480.00,19373.00,19167.00
Gross Margin,29681.00,27769.00,29440.00


In [15]:
ticker_list = list(master_data.keys())

for ticker in drop_list:
    
    try:
       ticker_list.remove(ticker)
    except ValueError:
       pass

ticker_list

['AA',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACHC',
 'ACI',
 'ACM',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AES',
 'AFG',
 'AFL',
 'AFRM',
 'AGCO',
 'AGL',
 'AGNC',
 'AGO',
 'AGR',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'AL',
 'ALB',
 'ALGM',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALLY',
 'ALNY',
 'ALSN',
 'AM',
 'AMAT',
 'AMC',
 'AMCR',
 'AMD',
 'AME',
 'AMED',
 'AMG',
 'AMGN',
 'AMP',
 'AMZN',
 'AN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APO',
 'APP',
 'APTV',
 'AR',
 'ARES',
 'ARMK',
 'ARW',
 'ASH',
 'ATO',
 'ATR',
 'ATUS',
 'ATVI',
 'AVGO',
 'AVT',
 'AVTR',
 'AWI',
 'AWK',
 'AXON',
 'AXS',
 'AXTA',
 'AYI',
 'AYX',
 'AZEK',
 'AZO',
 'AZTA',
 'BA',
 'BAC',
 'BAH',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BC',
 'BDX',
 'BEN',
 'BERY',
 'BFAM',
 'BG',
 'BHF',
 'BIIB',
 'BILL',
 'BIO',
 'BJ',
 'BK',
 'BKNG',
 'BLD',
 'BLDR',
 'BLK',
 'BMRN',
 'BMY',
 'BOKF',
 'BR',
 'BRKR',
 'BRO',
 'BSX',
 'BSY',
 'BURL',
 'BWA',
 'BW

In [16]:
len(ticker_list)

833

### Remove any Companys with less than 8 years of data.

In [18]:
drop_ticker_list = []

ticker_list = list(master_data.keys())

for ticker in ticker_list:
  
  print(ticker)
  
  no_keys = len(master_data[ticker].keys())
    
  if no_keys < 8:
    print(no_keys)
    drop_ticker_list.append(ticker)

drop_ticker_list

AA
7
AAL
AAP
AAPL
ABBV
ABNB
3
ABT
ACGL
ACHC
ACI
6
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
6
AEE
AEP
AES
AFG
AFL
AFRM
2
AGCO
AGL
2
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
2
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
6
AMAT
AMC
AMCR
4
AMD
AME
AMED
AMG
AMGN
AMH
AMP
AMT
AMZN
AN
ANET
ANSS
AON
AOS
APA
2
APD
APH
APO
1
APP
2
APTV
AR
ARE
ARES
ARMK
ARW
ASH
7
ATO
ATR
ATUS
6
ATVI
AVB
AVGO
5
AVT
AVTR
4
AWI
AWK
AXON
6
AXS
AXTA
AYI
AYX
6
AZEK
3
AZO
AZTA
1
BA
BAC
BAH
BALL
BAX
BBWI
2
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
6
BIIB
BILL
3
BIO
BJ
5
BK
BKI
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BRX
BSX
BSY
3
BURL
BWA
BWXT
BX
BXP
BYD
C
CABO
CACC
CACI
CAG
CAH
CAR
CARR
3
CASY
CAT
CB
CBOE
CBRE
CBSH
CC
CCCS
2
CCI
CCK
CCL
CDAY
5
CDNS
CDW
CE
CEG
2
CERT
3
CF
CFG
CFLT
2
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHK
CHPT
2
CHRW
CHTR
CI
5
CIEN
CINF
CL
CLH
CLVT
3
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNC
CNM
2
CNP
CNXC
3
COF
COIN
2
COLB
COLD
6
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CPT
CR
CRI
CRL
CRUS
CRWD
4
CSCO
CSGP
CSL
CSX
CTAS
CTLT
CTRA
2
CTSH
CTVA
4

['AA',
 'ABNB',
 'ACI',
 'ADT',
 'AFRM',
 'AGL',
 'ALGM',
 'AM',
 'AMCR',
 'APA',
 'APO',
 'APP',
 'ASH',
 'ATUS',
 'AVGO',
 'AVTR',
 'AXON',
 'AYX',
 'AZEK',
 'AZTA',
 'BBWI',
 'BHF',
 'BILL',
 'BJ',
 'BSY',
 'CARR',
 'CCCS',
 'CDAY',
 'CEG',
 'CERT',
 'CFLT',
 'CHPT',
 'CI',
 'CLVT',
 'CNM',
 'CNXC',
 'COIN',
 'COLD',
 'CRWD',
 'CTRA',
 'CTVA',
 'CVNA',
 'CZR',
 'DASH',
 'DBX',
 'DD',
 'DDOG',
 'DH',
 'DINO',
 'DIS',
 'DKNG',
 'DNA',
 'DNB',
 'DOCS',
 'DOCU',
 'DOW',
 'DRVN',
 'DT',
 'DTM',
 'DV',
 'DXC',
 'EHAB',
 'ELAN',
 'ELV',
 'ENOV',
 'EQH',
 'ESAB',
 'ESTC',
 'ETSY',
 'EVRG',
 'FG',
 'FHB',
 'FND',
 'FOUR',
 'FOX',
 'FOXA',
 'FTV',
 'FYBR',
 'GO',
 'GTES',
 'GXO',
 'HAYW',
 'HR',
 'HTZ',
 'IAC',
 'INFA',
 'INVH',
 'IR',
 'JAMF',
 'JBGS',
 'JHG',
 'KD',
 'LCID',
 'LESL',
 'LUMN',
 'LW',
 'LYFT',
 'MCW',
 'MDB',
 'META',
 'MP',
 'MRNA',
 'MRVI',
 'MRVL',
 'MSGS',
 'NCNO',
 'NET',
 'NFE',
 'NTNX',
 'NVCR',
 'NVST',
 'NVT',
 'OGN',
 'OKTA',
 'OLPX',
 'OPEN',
 'OSH',
 'OTIS',
 'OVV

In [19]:
len(drop_ticker_list)

173

In [21]:
for ticker in drop_ticker_list:
    print(ticker)
    master_data.pop(ticker,None)

AA
ABNB
ACI
ADT
AFRM
AGL
ALGM
AM
AMCR
APA
APO
APP
ASH
ATUS
AVGO
AVTR
AXON
AYX
AZEK
AZTA
BBWI
BHF
BILL
BJ
BSY
CARR
CCCS
CDAY
CEG
CERT
CFLT
CHPT
CI
CLVT
CNM
CNXC
COIN
COLD
CRWD
CTRA
CTVA
CVNA
CZR
DASH
DBX
DD
DDOG
DH
DINO
DIS
DKNG
DNA
DNB
DOCS
DOCU
DOW
DRVN
DT
DTM
DV
DXC
EHAB
ELAN
ELV
ENOV
EQH
ESAB
ESTC
ETSY
EVRG
FG
FHB
FND
FOUR
FOX
FOXA
FTV
FYBR
GO
GTES
GXO
HAYW
HR
HTZ
IAC
INFA
INVH
IR
JAMF
JBGS
JHG
KD
LCID
LESL
LUMN
LW
LYFT
MCW
MDB
META
MP
MRNA
MRVI
MRVL
MSGS
NCNO
NET
NFE
NTNX
NVCR
NVST
NVT
OGN
OKTA
OLPX
OPEN
OSH
OTIS
OVV
OWL
PARA
PARAA
PATH
PCOR
PFGC
PINS
PK
PLTK
PLTR
PTON
PYCR
QDEL
QS
REYN
RRX
S
SHC
SITE
SMAR
SNDR
SNOW
SOFI
SSRM
STE
TNL
TOST
TPL
TTD
TW
TWKS
TWLO
TXG
U
UBER
UPST
USFD
UWMC
VICI
VNT
VRT
VSCO
VST
VTRS
VVV
WBD
WE
WH
WMS
WOLF
WOOF
WRK
WTW
YETI


In [23]:
# Generate a clean version of each table

clean_master_data = {}
ticker_list = list(master_data.keys())

for ticker in ticker_list:
  print(ticker)

  clean_master_data[ticker] = {}
    
  for yr in master_data[ticker]:
    
    clean_master_data[ticker][yr] = {}

    for stype in master_data[ticker][yr]:
    
      try:
        clean_master_data[ticker][yr][stype] = clean_table(master_data[ticker][yr][stype])
        
      except:
        print("Error : {} {} {}".format(ticker, yr, stype))
        exit(1)
      

AAL
AAP
AAPL
ABBV
ABT
ACGL
ACHC
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFG
AFL
AGCO
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AMAT
AMC
AMD
AME
AMED
AMG
AMGN
AMH
AMP
AMT
AMZN
AN
ANET
ANSS
AON
AOS
APD
APH
APTV
AR
ARE
ARES
ARMK
ARW
ATO
ATR
ATVI
AVB
AVT
AWI
AWK
AXS
AXTA
AYI
AZO
BA
BAC
BAH
BALL
BAX
BBY
BC
BDX
BEN
BERY
BFAM
BG
BIIB
BIO
BK
BKI
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BRX
BSX
BURL
BWA
BWXT
BX
BXP
BYD
C
CABO
CACC
CACI
CAG
CAH
CAR
CASY
CAT
CB
CBOE
CBRE
CBSH
CC
CCI
CCK
CCL
CDNS
CDW
CE
CF
CFG
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHK
CHRW
CHTR
CIEN
CINF
CL
CLH
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNC
CNP
COF
COLB
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CPT
CR
CRI
CRL
CRUS
CSCO
CSGP
CSL
CSX
CTAS
CTLT
CTSH
CUBE
CUZ
CVS
CVX
CW
D
DAL
DAR
DCI
DECK
DEI
DFS
DG
DGX
DHI
DHR
DISH
DKS
DLB
DLR
DLTR
DOV
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EEFT
EFX
EGP
EHC
EIX
EL
ELS
EMN
EMR
ENPH
ENTG
EOG
EPAM
EPR
EQIX
EQR
EQT
ERIE
ES
ESI
ESS
ETN
ETR
EVA
EVR
EW
EWBC
EXAS
EXC
EXEL
EXP
EXPD

In [24]:
len(clean_master_data)

763

In [25]:
df = clean_master_data['CSCO']['15']['income']['table']
df

,"Jul. 25, 2015","Jul. 26, 2014","Jul. 27, 2013"
line_item,,,
Revenue:,NaN,NaN,NaN
Product,37750.00,36172.00,38029.00
Service,11411.00,10970.00,10578.00
Total Revenue,49161.00,47142.00,48607.00
Cost Of Sales:,NaN,NaN,NaN
Product,15377.00,15641.00,15541.00
Service,4103.00,3732.00,3626.00
Total Cost Of Sales,19480.00,19373.00,19167.00
Gross Margin,29681.00,27769.00,29440.00


In [26]:
master_data['CSCO']['15']['income']

Consolidated Statements Of Operations - USD ($)  shares in Millions, $ in Millions  \
   Consolidated Statements Of Operations - USD ($)  shares in Millions, $ in Millions   
0                                                                            REVENUE:   
1                                                                             Product   
2                                                                             Service   
3                                                                       Total revenue   
4                                                                      COST OF SALES:   
5                                                                             Product   
6                                                                             Service   
7                                                                 Total cost of sales   
8                                                                        GROSS MARGIN   
9                                                                 OPERATING EXPENSES:   
10                                                           Research and development   
11                                                                Sales and marketing   
12                                                         General and administrative   
13                                        Amortization of purchased intangible assets   
14                                                    Restructuring and other charges   
15                                                           Total operating expenses   
16                                                                   OPERATING INCOME   
17                                                                    Interest income   
18                                                                   Interest expense   
19                                                           Other income (loss), net   
20                                              Interest and other income (loss), net   
21                                           INCOME BEFORE PROVISION FOR INCOME TAXES   
22                                                         Provision for income taxes   
23                                                                         NET INCOME   
24                                                              Net income per share:   
25                                                       Basic (in dollars per share)   
26                                                     Diluted (in dollars per share)   
27                                              Shares used in per-share calculation:   
28                                                                  Basic (in shares)   
29                                                                Diluted (in shares)   
30                    Cash dividends declared per common share (in dollars per share)   

   12 Months Ended                              
     Jul. 25, 2015 Jul. 26, 2014 Jul. 27, 2013  
0              nan           nan           nan  
1         $ 37,750      $ 36,172      $ 38,029  
2            11411         10970         10578  
3            49161         47142         48607  
4              nan           nan           nan  
5            15377         15641         15541  
6             4103          3732          3626  
7            19480         19373         19167  
8            29681         27769         29440  
9              nan           nan           nan  
10            6207          6294          5942  
11            9821          9503          9538  
12            2040          1934          2264  
13             359           275           395  
14             484           418           105  
15           18911         18424         18244  
16           10770          9345         11196  
17             769           691           654  
18           (566)         (564)         (583)  
19             228           243    

In [27]:
# Save to file

with open(PROJ_ROOT_PATH + '/pickle/clean_master_data_interim.pkl', 'wb') as f:
  pickle.dump(clean_master_data, f)


## Create List of Table Labels

In [20]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','clean_master_data_interim.pkl')
clean_master_data = pd.read_pickle(filepath)

In [22]:
def get_line_items(t):
 
  headings = []
  line_items = []
    
  for index, row in t.iterrows():
    
    if row.replace('', np.nan).isna().all():
      # Blank row, so assume heading
    
      headings.append(index)
    else:    
      line_items.append(index)

  return headings, line_items

In [23]:
def get_table_labels(stype):

  master_headings = []
  master_line_items = []

  ticker_list = list(clean_master_data.keys())

  for ticker in ticker_list:
    
    print(ticker)
    
    for yr in clean_master_data[ticker]:
      
      # Allow for the likelihood that statement might be missing for given ticker and yr
        
      if stype in clean_master_data[ticker][yr]:
        try:
          headings, line_items = get_line_items(clean_master_data[ticker][yr][stype]['table'])
    
          master_headings = master_headings + headings
          master_line_items = master_line_items + line_items
        except:
          print("Error : {} {}".format(ticker, yr))
          exit() 

  # Remove duplicates from headings and line items

  unique_list = set(master_headings)
  headings = list(unique_list)

  unique_list = set(master_line_items)
  line_items = list(unique_list)

  return {'headings' : headings,
          'line_items' : line_items}


In [28]:
# Save to file

with open(PROJ_ROOT_PATH + '/pickle/clean_master_data_stage2.pkl', 'wb') as f:
  pickle.dump(clean_master_data, f)